# Lab 10 Prelab - Preparing to reanalyze your Lab 09 data

In [1]:
%reset -f
import data_entry2
import numpy as np
import matplotlib.pyplot as plt

In Lab 09, it is likely that you encountered some unexpected behaviour in your data due to background radiation. And this background radiation likely posed significant challenges when trying to fit a linear model to `Ln(R)` data. In Lab 10, in additional to improving your data set, you will be reanlyzing your Lab 09 data with a new nonlinear model that accounts for this background radiation.

Unfortunately, we don't have nice analytic solutions that can handle the new nonlinear model we will be using, so we will be revisiting some of our fitting by hand, with the help of residuals plots and chi-squared. To prepare for this, you are going to adapt the 2-parameter fit code from `Resources/Lab06-demo.ipynb`.

### Your turn #1

Below we provide some starting code for you to copy in your Lab 09 data and perform the linear 2-parameter (i.e., y = mx+ b) fit on these data without the analytic solutions. Modify the code below so that it can be used to fit your Lab 09 data. Update your plots so they have appropriate title, axis labels and legends. To ensure your code is working as nitended, you can use the same slope and y-intercept that you found for your best fit in Lab 09. 

**Reminder:** During the lab we are going to be fitting these data to a new nonlinear model "by hand", meaming where you will be adjusting the parameters to try to find the minimum possible chi-squared. This exercise is to help you have relevant code available to do this.

In [2]:
# Make a copy of your lab 09 data. 
# - You may need to update "lab09_data" to match what you named the spreadsheet
de1 = data_entry2.sheet_copy("../Lab09/lab09_data", "lab09_data-copy")

Sheet name: lab09_data-copy.csv


In [2]:
# Additional support code as needed



In [ ]:
### Code from Resources/Lab06-demo.ipynb Appendix B: Including a y-intercept in your model

# Make the plots and calculate chi-squared

# Scatter step 1: Define the variables we will be plotting, as well as labels and titles
# Plotting variables
xdata = DxVec
ydata = FVec
dydata = dFVec

# Labels and titles
data_label = "Experimental data"
model_label = "F = kx + b" ### Included a y-intercept
graph_title = "Hooke's law investigation using spring compression"
x_label = "Displacement of spring from equilibrium (m)"
y_label = "Force (N)"
residuals_title = "Residuals for Hooke's law investigation using spring compression"
residuals_y_label = "Residual = data - model (N)"

# Model parameters
slope = 2.09 # The best slope from the y = mx model
intercept = 0. ### Added the y-intercept, b
P = 2 ### Updated number of fitting parameters to be 2 (m and b)


# Scatter step 2: find the limits of the data:
xmin = np.min(xdata) # use the np.min function to find the smallest x-value
xmax = np.max(xdata) # same for max
# print (xmin, xmax)  # uncomment to see what the limits are

# Scatter step 3: generate a bunch of x points between xmin and xmax to help us plot the model line
xpoints = np.linspace(xmin, xmax, 200) # gives 200 evenly spaced points between xmin and xmax
# print(xpoints) # uncomment to see the x values that were generated.

# Scatter step 4: calculate the model values:
ypoints = slope * xpoints + intercept ### Update the model line to include the y-intercept

# Scatter step 5: plot the model line. We plot this as a red line "r-" :
plt.plot(xpoints, ypoints, "r-", label = model_label)

# Scatter step 6: Plot the data, with the previous details from before
plt.errorbar(xdata, ydata, dydata, fmt="bo", markersize = 3, label=data_label)
plt.title(graph_title)
plt.xlabel(x_label)
plt.ylabel(y_label)
plt.legend()
plt.show()

# Residuals step 1: Calculate the model prediction for each our data points from dxVec
ymodel = slope * xdata + intercept ### Updated model values at the data points to y = mx + b

# Residuals step 2: Calcualte the residuals vector
residualsVec = ydata - ymodel

# Residuals step 3: Plot the residuals vector against the x-data vector
plt.errorbar(xdata, residualsVec, dydata, fmt="bo", markersize = 3)

# Residuals step 4: Add a horizontal line at R=0 to the plot
plt.hlines(y=0, xmin=xmin, xmax=xmax, color='k') # draw a black line at y = 0.

# Residuals step 5: Add axis labels and title, and show the graph
plt.title(residuals_title)
plt.xlabel(x_label)
plt.ylabel(residuals_y_label)
plt.show()

# Calculate chi-squared 
chi2 = np.sum((residualsVec/dydata)**2)/(len(residualsVec)-P)
print ("Slope: ", slope, "N/m")
print ("Weighted chi-squared: ", chi2)

### Your turn #2

Once all of your code above is working, copy it over to your Lab 10 notebook so it is ready to be modified further for the Lab 10 analysis. Notice that we have included some code in your Lab 10 notebook to help you keep track of your values for chi-squared as you modify your parameters.

# Submit

Steps for submission:

1. Click: Run => Run_All_Cells
2. Read through the notebook to ensure all the cells executed correctly and without error.
3. File => Save_and_Export_Notebook_As->HTML
4. Inspect your html file
5. Upload the HTML document to the lab submission assignment on Canvas.